### Step 1: Prepare documents

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
        

In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Step 2: Create Embeddings using Pretrained Models

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
len(model.encode('This is a simple sentence'))

768

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
from tqdm.autonotebook import tqdm

operations = []

for doc in tqdm(documents):
    doc['text_vector'] = list(model.encode(doc['text']))
    operations.append(doc)

100%|██████████| 948/948 [05:00<00:00,  3.16it/s]


In [9]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346,
  -0.06891429,
  -0.044483423,
  0.0062501533,
  -0.042247094,
  -0.0064539956,
  0.027548144,
  -0.018610273,
  -0.052037504,
  -0.0024047394,
  0.0574545,
  -0.0155926,
  0.015488839,
  0.0987016,
  0.015649557,
  -0.061262548,
  0.04107641,
  0.021717312,
  -0.09465402,
  0.0032483656,
  0.0044765803,
  -0.0076589384,
  -0.05192878,
  -0.007335758

### Step 3: Setup ElasticSearch connection

In [10]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [11]:
es_client.info()

ObjectApiResponse({'name': 'ca450a10ec81', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gc1YitWtRBu57KZdgqTclQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [13]:

index_name = "course-questions"

# es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
search_term = "windows or mac?"

vector_search_term = model.encode(search_term)

In [16]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates":10000
}

In [17]:
res = es_client.search(index=index_name,knn=query,source=["text", "section", "question", "course"])
res['hits']['hits']

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7b3ca16e8400>: Failed to establish a new connection: [Errno 111] Connection refused))